In [9]:
class SegmentTree:
    def __init__(self, n):
        self.tree = [0] * (4 * n)  # Approximate size for segment tree
        self.lazy = [0] * (4 * n)  # Lazy propagation array

    def update_range(self, node, start, end, l, r, val):
        if self.lazy[node] != 0:
            # Update current node due to previous pending updates
            self.tree[node] += (end - start + 1) * self.lazy[node]
            if start != end:  # Not a leaf node
                self.lazy[node * 2] += self.lazy[node]
                self.lazy[node * 2 + 1] += self.lazy[node]
            self.lazy[node] = 0
        if start > end or start > r or end < l:  # No overlap
            return
        if start >= l and end <= r:  # Total overlap
            self.tree[node] += (end - start + 1) * val
            if start != end:  # Not a leaf node
                self.lazy[node * 2] += val
                self.lazy[node * 2 + 1] += val
            return
        mid = (start + end) // 2
        self.update_range(node * 2, start, mid, l, r, val)
        self.update_range(node * 2 + 1, mid + 1, end, l, r, val)
        self.tree[node] = self.tree[node * 2] + self.tree[node * 2 + 1]

    def query_range(self, node, start, end, l, r):
        if start > end or start > r or end < l:
            return 0
        if self.lazy[node] != 0:
            self.tree[node] += (end - start + 1) * self.lazy[node]
            if start != end:
                self.lazy[node * 2] += self.lazy[node]
                self.lazy[node * 2 + 1] += self.lazy[node]
            self.lazy[node] = 0
        if start >= l and end <= r:
            return self.tree[node]
        mid = (start + end) // 2
        p1 = self.query_range(node * 2, start, mid, l, r)
        p2 = self.query_range(node * 2 + 1, mid + 1, end, l, r)
        return p1 + p2

In [2]:
def count__intersections_with_segment_tree(radians, identifiers):
    # Map each radian to a unique integer
    unique_radians = sorted(set(radians))
    radian_to_index = {rad: idx for idx, rad in enumerate(unique_radians)}

    # Initialize the segment tree
    segment_tree = SegmentTree(len(unique_radians))

    # Prepare a list of tuples (radian, is_start, chord_id) for sorting
    events = []
    for radian, identifier in zip(radians, identifiers):
        if '_' in identifier:
            # Correctly parse identifiers in the "s_x"/"e_x" format
            prefix, num = identifier.split('_')
            is_start = prefix == 's'
            chord_id = int(num)
        else:
            # Correctly parse identifiers in the "s1"/"e1" format
            is_start = identifier[0] == 's'
            chord_id = int(identifier[1:])

        events.append((radian_to_index[radian], is_start, chord_id))

    # Sort events first by radian index, then start points before end points
    events.sort(key=lambda x: (x[0], not x[1]))

    intersections = 0
    active_chords = {}  # Track active chords by their ID

    for radian_idx, is_start, chord_id in events:
        if is_start:
            # If it's the start of a chord, add it to the active chords
            active_chords[chord_id] = radian_idx
            # Update the segment tree and query for intersections
            segment_tree.update_range(1, 0, len(unique_radians) - 1, radian_idx, radian_idx, 1)
            intersections += segment_tree.query_range(1, 0, len(unique_radians) - 1, active_chords[chord_id], radian_idx) - 1
        else:
            # If it's the end of a chord, remove it from active chords and update the segment tree
            start_idx = active_chords.pop(chord_id)
            segment_tree.update_range(1, 0, len(unique_radians) - 1, start_idx, radian_idx, -1)

    return intersections

In [10]:
def count_intersections_with_segment_tree(radians, identifiers):
    # Map each radian to a unique integer
    unique_radians = sorted(set(radians))
    radian_to_index = {rad: idx for idx, rad in enumerate(unique_radians)}

    # Initialize the segment tree
    segment_tree = SegmentTree(len(unique_radians))

    # Prepare a list of tuples (radian, is_start, chord_id) for sorting
    events = []
    for radian, identifier in zip(radians, identifiers):
        if '_' in identifier:
            prefix, num = identifier.split('_')
            is_start = prefix == 's'
            chord_id = int(num)
        else:
            is_start = identifier[0] == 's'
            chord_id = int(identifier[1:])

        events.append((radian_to_index[radian], is_start, chord_id))

    # Sort events first by radian index, then start points before end points
    events.sort(key=lambda x: (x[0], not x[1]))

    intersections = 0
    active_chords = {}

    for radian_idx, is_start, chord_id in events:
        if is_start:
            # If it's the start of a chord, query for intersections before updating
            if chord_id in active_chords:
                start_idx = active_chords[chord_id]
                intersections += segment_tree.query_range(1, 0, len(unique_radians) - 1, start_idx, radian_idx)
            # Update the segment tree to add the chord
            segment_tree.update_range(1, 0, len(unique_radians) - 1, radian_idx, radian_idx, 1)
            active_chords[chord_id] = radian_idx
        else:
            # If it's the end of a chord, remove it from active chords and update the segment tree
            if chord_id in active_chords:
                start_idx = active_chords.pop(chord_id)
                segment_tree.update_range(1, 0, len(unique_radians) - 1, start_idx, radian_idx, -1)

    return intersections


In [11]:
# Example usage (assuming the SegmentTree class is implemented as previously described)
radians = [0.78, 1.47, 1.77, 3.92]
identifiers = ["s_1", "s_2", "e_1", "e_2"]
intersections = count_intersections_with_segment_tree(radians, identifiers)
print(f"Number of intersections: {intersections}")

Number of intersections: 0
